In [ ]:
# искать тексты примерно по 50-100 слов, надо будет посмотреть максимальный размер который поместиться в батч (50 слов в тестовом датасете)
# лучше всего парсить литературу

In [ ]:
!pip install transformers

In [2]:
import pandas as pd
import string
import re

from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
import torch

import nltk
from nltk.tokenize import word_tokenize
import random
import numpy as np
from tqdm import tqdm_notebook, tqdm

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import os
import pickle
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
os.chdir('/content/drive/My Drive/Huawei_for_job')

In [ ]:
#data = pd.read_csv('/content/drive/My Drive/Huawei/data_rio.csv')
#data = data.dropna()

In [ ]:
#clean = data['text'].apply(lambda x: len(word_tokenize(str(x)))>70).values
#data = data[clean]

In [4]:
with open('./dataset.txt', 'r', encoding='utf-8') as f:
  data = ''.join(f.readlines())

In [5]:
data_tokens = word_tokenize(data)

In [8]:
with open('data_tokens.pickle', 'wb') as f:
  pickle.dump(data_tokens, f)

In [7]:
with open('data_tokens.pickle', 'rb') as f:
  data_tokens = pickle.load(f)

In [ ]:
data_tokens

In [62]:
len(data_tokens)

29380485

In [6]:
def get_dataset(data, size = 300, download = True):
  if download:
    with open('data_tokens.pickle', 'rb') as f:
        data_tokens = pickle.load(f)
  else:
    data_tokens = word_tokenize(data)

  dataset = []
  for i in tqdm_notebook(range(0, len(data_tokens), size)):
    dataset.append(' '.join(data_tokens[i: i+size]))
  return dataset

In [7]:
dataset = get_dataset(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [69]:
len(dataset)

97935

In [70]:
"""X = data['text'].apply(lambda x: ' '.join(filter(lambda x: x not in punctuation, word_tokenize(str(x)))))
y = data['text'].apply(lambda x: str(x).lstrip())
X = X.values
y = y.values"""

"X = data['text'].apply(lambda x: ' '.join(filter(lambda x: x not in punctuation, word_tokenize(str(x)))))\ny = data['text'].apply(lambda x: str(x).lstrip())\nX = X.values\ny = y.values"

In [77]:
#np.save('/content/drive/My Drive/y', y)
#np.save('/content/drive/My Drive/X', X)

In [78]:
#X = np.load('./X.npy', allow_pickle=True)
#y = np.load('./y.npy', allow_pickle=True)

In [8]:
X_train, X_test = train_test_split(dataset, test_size = 0.1)

In [ ]:
#from transformers import BertTokenizer, BertForMaskedLM

#tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
#PAD = 0
#MASK = 103
#CLS = 101
#SEP = 102
#UNK = 100

In [9]:
class Tokenizer:
  def __init__(self, full_data, vocab_size, lower = False, download = True):
    if lower:
      full_data = full_data.lower()

    self.vocab_size = vocab_size
    self.lower = lower
    if download:
      with open('tokenizer_vocab.pickle', 'rb') as f:
        self.vocab = pickle.load(f)
    else:
        tokens = word_tokenize(full_data, language='russian')
        self.vocab = pd.Series(nltk.FreqDist(tokens)).sort_values(ascending=False)[:self.vocab_size-1].index

    self.converter = preprocessing.LabelEncoder()
    self.converter.fit(self.vocab.tolist() + ['unk'])
    self.classes = self.converter.classes_
    self.unk = self.converter.transform(['unk'])[0]
  
  def convert_to_ids(self, input):
    if self.lower:
      input = input.lower()

    input = word_tokenize(input)
    ids = []
    for word in input:
      if word in self.classes:
        ids += self.converter.transform([word]).tolist()
      else:
        ids += [self.unk]
    
    return ids

  def convert_to_subwords(self, ids):
    return self.converter.inverse_transform(ids)

In [10]:
tokenizer = Tokenizer(data, 50000)

In [17]:
tokenizer.convert_to_ids('vdjfvn.lcm;se')

[1171, 53, 1030]

In [18]:
#сделать поменьше список только из самых нужных знаков препинания
punctuation = string.punctuation
punctuation_set = set('!,.-?:;')
#punctuation = set(string.punctuation)
nothing = ''
#padding = 'PAD'
labels = [nothing] + list(punctuation_set)

le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [19]:
from transformers import BertConfig

config = BertConfig(
                    vocab_size=tokenizer.vocab_size,
                    encoder_layers = 3,
                    decoder_layers = 3
                    )

In [12]:
from transformers import BertForMaskedLM

class BertPunc(nn.Module):  
    
    def __init__(self, dropout, seq_len = 50):
        super(BertPunc, self).__init__()
        self.bert = BertForMaskedLM(config)
        self.bert_vocab_size = tokenizer.vocab_size
        self.bn = nn.BatchNorm1d(seq_len)
        self.fc = nn.Linear(self.bert_vocab_size, len(labels))
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids = None, attention_mask = None, token_type_ids = None):
        x = self.bert(input_ids = input_ids,
                      attention_mask = attention_mask,
                      token_type_ids = token_type_ids)
        x = self.fc(self.dropout(self.bn(x[0])))
        return x

In [75]:
with open('target_train.pickle', 'wb') as f:
  dataset = []
  for row in X_train:
    dataset.append(word_tokenize(row))
  pickle.dump(dataset, f)

In [76]:
with open('target_test.pickle', 'wb') as f:
  dataset = []
  for row in X_test:
    dataset.append(word_tokenize(row))
  pickle.dump(dataset, f)

In [77]:
with open('data_train.pickle', 'wb') as f:
  dataset = []
  for row in X_train:
    dataset.append(' '.join(list(filter(lambda x: x not in punctuation, word_tokenize(row)))))
  pickle.dump(dataset, f)

In [78]:
with open('data_test.pickle', 'wb') as f:
  dataset = []
  for row in X_test:
    dataset.append(' '.join(list(filter(lambda x: x not in punctuation, word_tokenize(row)))))
  pickle.dump(dataset, f)

In [13]:
with open('data_train.pickle', 'rb') as f:
  data = pickle.load(f)

In [14]:
with open('target_test.pickle', 'rb') as f:
  target = pickle.load(f)

In [53]:
def r():
  x = tokenizer.convert_to_ids(data[0])[:50]
  token_type, attn = [0]*len(x), [1]*len(x)
  tokens = target[0]
  labels = []
  j = 0
  i = 0
  while i < len(tokens)-1:
    if tokens[i+1] in punctuation_set:
      labels.append(tokens[i+1])
      i += 2
    else:
      labels.append('')
      i += 1  
  labels = le.transform(labels).tolist()[:50]
  x = torch.tensor(x, dtype = torch.long)
  attention_mask = torch.tensor(attn)
  labels = torch.tensor(labels, dtype = torch.long)
  token_type = torch.tensor(token_type)

In [83]:
def punc_check(data):
  dataset = []
  for row in data:
    dataset.append(' '.join(list(filter(lambda x: x not in punctuation, word_tokenize(row)))))
  return dataset

class Punctuation(Dataset):
  def __init__(self, X, max_len = 50, mode = 'train'):
    #self.data = punc_check(X)
    self.max_len = max_len
    if mode == 'train':
      with open('target_train.pickle', 'rb') as f:
        self.target = pickle.load(f)
      
      with open('data_train.pickle', 'rb') as f:
        self.data = pickle.load(f)

      
    else:
      with open('target_test.pickle', 'rb') as f:
        self.target = pickle.load(f)

      with open('data_test.pickle', 'rb') as f:
        self.data = pickle.load(f)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    indexes = tokenizer.convert_to_ids(' '.join(word_tokenize(self.data[idx])[:self.max_len]))
    if len(indexes) == self.max_len:
      x = indexes
    else:
      x = tokenizer.convert_to_ids(self.data[idx])[:self.max_len]
    token_type, attn = [0]*len(x), [1]*len(x)
    tokens = self.target[idx]
    labels = []
    i = 0

    while i < len(tokens)-1:
      if tokens[i+1] in punctuation_set:
        labels.append(tokens[i+1])
        i += 2
      else:
        labels.append('')
        i += 1
    
    labels = le.transform(labels).tolist()[:self.max_len]
    if len(labels) != len(x) != self.max_len:
      print(self.data[idx])
      print(len(labels))
    assert len(labels) == len(x) == self.max_len
    
    x = torch.tensor(x, dtype = torch.long)
    attention_mask = torch.tensor(attn)
    labels = torch.tensor(labels, dtype = torch.long)
    token_type = torch.tensor(token_type)
    #tokens = ' '.join(tokens[:self.max_len])

    return x, attention_mask, token_type, labels

In [ ]:
"""def collate_fn(records):
  max_len_input = 52
  max_len_target = 50
  attention_mask = torch.zeros(len(records), max_len_input)
  target = torch.ones(len(records), max_len_target)*int(le.transform(['PAD']))
  input = torch.zeros(len(records), max_len_input)
  token_types = torch.zeros(len(records), max_len_input)

  for i, record in enumerate(records):
    len_ = min(max_len_input, len(record['input']))
    input[i, :len_] += record['input'][:len_]

    target[i][target[i]==0] = 1
    target[i, :len(record['labels'])] += record['labels']
    target[i][target[i]==24] = 0

    
    attention_mask[i, :len_] += record['attention_mask'][:len_]

  return {'input_ids':input,
          'attention_mask':attention_mask,
          'token_type_ids':token_types,
          'decoder_input_ids':target}"""

In [84]:
train_dataset = Punctuation(X_train)
#inp, attn, types, lab = next(iter(DataLoader(train_dataset, batch_size=100, shuffle=True)))

In [85]:
inp, attn, types, lab = next(iter(DataLoader(train_dataset, batch_size=3, shuffle=True)))

In [86]:
inp.shape, lab.shape

(torch.Size([3, 50]), torch.Size([3, 50]))

In [ ]:
from utils import check_one

def check_metric(input, attn, types, reference):
  model.eval()
  hypothesis_1 = le.inverse_transform(model(input, attn, types).squeeze().argmax(dim=-1))
  hypothesis = tokenizer.convert_ids_to_tokens(inp.squeeze())

  i = 0
  j = 0
  while i<len(hypothesis):
    hypothesis.insert(i+1, hypothesis_1[j])
    i += 2
    j += 1
  
  hypothesis = ' '.join(hypothesis)

  return check_one(reference[0], hypothesis)

def check_metric_epoch(inputs, attns, types, references):
  metrics = [check_metric(*batch) for batch in zip(inputs, attns, types, references)]
  return torch.tensor(metrics)

In [ ]:
check_metric_epoch(inp, attn, types, ref)

In [87]:
def train(model, dataloader, optimizer, criterion, clip):
    
    model.train()
    epoch_loss = 0.
    metrics = 0.
    with tqdm(desc="epoch", total=len(dataloader)) as pbar_outer:
      for batch in dataloader:

          optimizer.zero_grad()     

          x, attn, types, labels = batch    
          x = x.to(device)
          attn = attn.to(device)
          types = types.to(device)
          labels = labels.to(device)             
          
          outputs = model(x, attn, types)
          
          loss = criterion(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
          
          loss.backward()
          
          torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
          
          optimizer.step()
          
          epoch_loss += loss.item()

          pbar_outer.update(1)
          tqdm.write("train_loss:{}".format(loss.item()))
        
    train_loss = round( (epoch_loss / len(dataloader)), 3)
        
    return train_loss

In [88]:
def evaluate(model, dataloader, criterion):
    
    model.eval()
    epoch_loss = 0
    metrics = 0.
    
    with torch.no_grad():
      with tqdm(desc="epoch", total=len(dataloader)) as pbar_outer:
        for batch in tqdm_notebook(dataloader, total=len(dataloader)):

            x, attn, types, labels = batch    
            x = x.to(device)
            attn = attn.to(device)
            types = types.to(device)
            labels = labels.to(device)             
            
            outputs = model(x, attn, types)
        
            loss = criterion(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            
            epoch_loss += loss.item()
            
            pbar_outer.update(1)
            tqdm.write("val_loss:{}".format(loss.item()))
            
        valid_loss = round((epoch_loss / len(dataloader)), 3)
  
    return valid_loss

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt


device = torch.device('cuda')
#metrics = -100

model = BertPunc(0.3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-5)
train_dl = DataLoader(Punctuation(X_train), batch_size=16)
val_dl = DataLoader(Punctuation(X_test), batch_size=16)

clip = 3
num_epochs = 50
best_loss = 100
history_val = []
history_train = []

for epoch in tqdm_notebook(range(num_epochs)):
  
  train_loss = train(model, train_dl, optimizer, criterion, clip)
  valid_loss = evaluate(model, val_dl, criterion)

  history_val.append(valid_loss)
  history_train.append(train_loss)

  clear_output(True)
  plt.plot(history_train, label = 'train_loss')
  plt.plot(history_val, label = 'valid_loss')
  plt.legend()
  plt.show()

  print('Epoch: {} \n Train Loss {}  Val loss {}'.format(epoch + 1, train_loss, valid_loss))

  if valid_loss < best_loss:
    best_loss = valid_loss
    torch.save(model.state_dict(), 'puncBert.pth')

In [90]:
model(inp.to(device), attn.to(device), types.to(device)).argmax(dim=-1)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], device='cuda:0')

In [91]:
lab

tensor([[0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 4, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0,
         2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 7, 2, 0, 0, 0, 0, 4, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 0, 0, 0,
         0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0,
         0, 2]])

In [64]:
st = 'ете “ был хуже .Читательница Наталья Душина из г. Кологрива Костромской губернии приславшая Чехову в г. несколько восторженных писем писала ему в марте г. по поводу достоверности чеховских персонажей И „Муж “ в Вашем рассказе мне знаком тупой злой и страшный .Н К. Михайловский после выхода I тома издания А. Ф. Маркса отметив однообразие помещенных там рассказов молодого Чехова забавно и добродушно-примирительно излагающих отнюдь не смешные ситуации преимущественно супружеской жизни выделил из них только рассказ Муж как нечто довольно исключительное в сборнике г. Чехова — исключительное не по сюжету а по тону каким говорит автор Здесь по словам Михайловского автор и сам не смеется и не желает возбуждать смех в читателе Читатель переполняется ощущением жути и воспринимает героев как жертвы беспробудной пошлости жизни Принципиально иное отношение Чехова к изображаемому по мнению Михайловского обусловило в Муже и высокую степень истинно художественного творчества художественные приемы Чехова в этом рассказе показались критику более тонкими по сравнению с другими и он расценил его как настоящий перл в художественном отношении Выдающимися сценами по манере изображения он считал пляску Анны Павловны с черным неуклюжим и скуластым офицером кроме танцев ни на что не пригодным и затем изображение разгорающегося недовольства мужа переходящего в гнев по поводу веселости жены верхом совершенства представилась Михайловскому заключительная сцена в собрании когда Анна Павловна шёпотом умоляет мужа остаться еще на этом танцевальном вечере Эту поэзию контрастов и нотки гнева или хотя бы недовольства Михайловский не видел в других рассказах на сходные темы от которых Муж отличается более глубоким настроением Михайловский находил что тенденции проявившиеся в Муже со временем набирают силу и всё чаще'
